This Dataset is from: https://www.kaggle.com/datasets/anlgrbz/student-demographics-online-education-dataoulad/discussion/211264?resource=download

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [29]:
studentInfo = pd.read_csv("./data/studentInfo.csv")
studentInfo.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass


In [30]:
studentInfo['age_band'].dtype

dtype('O')

In [31]:
studentInfo.isna().sum()

code_module                0
code_presentation          0
id_student                 0
gender                     0
region                     0
highest_education          0
imd_band                1111
age_band                   0
num_of_prev_attempts       0
studied_credits            0
disability                 0
final_result               0
dtype: int64

In [32]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix



In [33]:
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'), ['highest_education'])
    ,('onehot2', OneHotEncoder(handle_unknown='ignore'), ['age_band'])],
    remainder='passthrough'
)

In [34]:
studentInfo.describe()

,id_student,num_of_prev_attempts,studied_credits
count,3.259300e+04,32593.000000,32593.000000
mean,7.066877e+05,0.163225,79.758691
std,5.491673e+05,0.479758,41.071900
min,3.733000e+03,0.000000,30.000000
25%,5.085730e+05,0.000000,60.000000
50%,5.903100e+05,0.000000,60.000000
75%,6.444530e+05,0.000000,120.000000
max,2.716795e+06,6.000000,655.000000


In [35]:
highest_education = studentInfo['highest_education']
c = set()
for i in highest_education:
    c.add(i)

print(c)

{'A Level or Equivalent', 'Lower Than A Level', 'Post Graduate Qualification', 'HE Qualification', 'No Formal quals'}


In [36]:
#gender,highest_education,studied_credits,disability,final_reasult
data = studentInfo[['gender', 'highest_education','age_band', 'studied_credits', 'disability','final_result']]
data.head()


,gender,highest_education,age_band,studied_credits,disability,final_result
0,M,HE Qualification,55<=,240,N,Pass
1,F,HE Qualification,35-55,60,N,Pass
2,F,A Level or Equivalent,35-55,60,Y,Withdrawn
3,F,A Level or Equivalent,35-55,60,N,Pass
4,F,Lower Than A Level,0-35,60,N,Pass


In [37]:
le = LabelEncoder()
data['gender'] = le.fit_transform(data['gender'])
data['disability'] = le.fit_transform(data['disability'])


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_24280\4172702738.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['gender'] = le.fit_transform(data['gender'])
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_24280\4172702738.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['disability'] = le.fit_transform(data['disability'])


In [38]:
data.head()

,gender,highest_education,age_band,studied_credits,disability,final_result
0,1,HE Qualification,55<=,240,0,Pass
1,0,HE Qualification,35-55,60,0,Pass
2,0,A Level or Equivalent,35-55,60,1,Withdrawn
3,0,A Level or Equivalent,35-55,60,0,Pass
4,0,Lower Than A Level,0-35,60,0,Pass


In [39]:
x = data[['gender', 'highest_education', 'studied_credits','age_band', 'disability']]
y = data['final_result']
#y = [y[i] for i in range(len(y))]
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=66)


In [40]:
clf = Pipeline([
    ('preprocessor', preprocessor),
    ('DecisionTreeClassifier', DecisionTreeClassifier(max_depth=3, min_samples_split=4, min_samples_leaf=4, criterion='entropy'))
])


In [41]:
clf.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['highest_education']),
                                                 ('onehot2',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['age_band'])])),
                ('DecisionTreeClassifier',
                 DecisionTreeClassifier(criterion='entropy', max_depth=3,
                                        min_samples_leaf=4,
                                        min_samples_split=4))])

In [42]:
y_pred = clf.predict(X_test)


In [43]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 42.14%


In [44]:
report = classification_report(y_test, y_pred)
print('Classification Report:')
print(report)

Classification Report:
              precision    recall  f1-score   support

 Distinction       0.00      0.00      0.00       583
        Fail       0.00      0.00      0.00      1402
        Pass       0.41      0.89      0.56      2533
   Withdrawn       0.48      0.25      0.33      2001

    accuracy                           0.42      6519
   macro avg       0.22      0.28      0.22      6519
weighted avg       0.31      0.42      0.32      6519



c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [45]:
pip2 = Pipeline([
    ('preprocessor', preprocessor), ('lr', LogisticRegression())
])
pip2

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['highest_education']),
                                                 ('onehot2',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['age_band'])])),
                ('lr', LogisticRegression())])

In [46]:
pip2.fit(X_train, y_train)

c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['highest_education']),
                                                 ('onehot2',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['age_band'])])),
                ('lr', LogisticRegression())])

In [47]:
y_pred = pip2.predict(X_test)

In [48]:
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 42.11%


In [49]:
report = classification_report(y_test, y_pred)
print('Classification Report:')
print(report)

c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report:
              precision    recall  f1-score   support

 Distinction       0.00      0.00      0.00       583
        Fail       0.00      0.00      0.00      1402
        Pass       0.42      0.79      0.55      2533
   Withdrawn       0.43      0.38      0.40      2001

    accuracy                           0.42      6519
   macro avg       0.21      0.29      0.24      6519
weighted avg       0.29      0.42      0.34      6519



c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
